# Классификация текстов: спам-фильтр для SMS

В настоящем проекте мы взяли открытый датасет с SMS-сообщениями, размеченными на спам ("spam") и не спам ("ham"), построили на нем классификатор текстов на эти два класса, оценили его качество посредством F1-меры, и посмотрели, что будет происходить с качеством, если менять параметры модели.

1. Загружаем датасет. Описание датасета можно посмотреть здесь: https://www.kaggle.com/uciml/sms-spam-collection-dataset.

2. Считываем датасет в Python, выясняя, что используется в качестве разделителей и как проставляются метки классов. Смотрим, как выглядят типичные сообщения обоих классов.

In [1]:
import re

file = open('SMSSpamCollection.txt', "r", encoding="utf-8").read()

print(re.search('ham\t(.+)\n', file).group(0))
print(re.search('spam\t(.+)\n', file).group(0))

ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...

spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's



3. Делим сообщения на 2 класса "спам" и "не спам", используя регулярные выражения. Создаем для дальнейшей работы список соответствующих меток классов. В качестве метки класса использовали 1 для спама и 0 для "не спама". Смотрим на (не)сбалансированность классов.
4. Разделяем датасет на обучающую и тестовую выборки в пропорции 80/20.

In [2]:
from sklearn.model_selection import train_test_split

ham = re.findall('ham\t(.+)\n', file)
spam = re.findall('spam\t(.+)\n', file)

print('Размер выборки:', len(ham+spam))
print('Количество не спама:', len(ham))
print('Количество спама:', len(spam))

data = ham + spam
labels = [0]*len(ham) + [1]*len(spam)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)

Размер выборки: 5573
Количество не спама: 4826
Количество спама: 747


5. Используя CountVectorizer со стандартными настройками, получаем из текстов матрицу признаков X.
6. Классифицируем тексты с помощью LogisticRegression с параметрами по умолчанию, делаем предсказания и оцениваем их качество с помощью F1-меры.

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score

vectorizer = CountVectorizer()
logreg = LogisticRegression(solver='lbfgs')

clf_pipeline = Pipeline(
            [("vectorizer", vectorizer), ("classifier", logreg)])
clf_pipeline.fit(X_train, y_train)

y_pred = clf_pipeline.predict(X_test)
print(f1_score(y_test, y_pred))

0.9466666666666667


7. Повторяем шаги из п.4 и п.5 для TfidfVectorizer. И замечаем, что качество ухудшается. Учитывая, что тексты не были предобаботаны (удаление знаков препинания и лемматизация слов), могла получиться довольна разреженная матрица признаков.

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

clf_pipeline = Pipeline(
            [("vectorizer", vectorizer), ("classifier", logreg)])
clf_pipeline.fit(X_train, y_train)

y_pred = clf_pipeline.predict(X_test)
print(f1_score(y_test, y_pred))

0.8975265017667845


8. Задаем в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измеряем получившееся значение F1-меры. В данном эксперименте мы пробуем добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратим внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [5]:
for var in [(2, 2), (3, 3), (1, 3)]:
    vectorizer = CountVectorizer(ngram_range=var)
    
    clf_pipeline = Pipeline(
            [("vectorizer", vectorizer), ("classifier", logreg)])
    clf_pipeline.fit(X_train, y_train)
    
    y_pred = clf_pipeline.predict(X_test)
    print(var, f1_score(y_test, y_pred))

(2, 2) 0.766798418972332
(3, 3) 0.6723404255319149
(1, 3) 0.9246575342465754


9. Вывод: наивысшее качество (94,67%) в нашей задачи классификация показывает CountVectorizer с использованием сочетания только униграмм в качестве признаков.

10. *Бонус: примеры сообщений для теста:

"FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB";

"FreeMsg: Txt: claim your reward of 3 hours talk time";

"Have you visited the last lecture on physics?";

"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$";

"Only 99$".

In [13]:
test = input('Ваше тестовое сообщение: ')

vectorizer = TfidfVectorizer()

clf_pipeline = Pipeline(
            [("vectorizer", vectorizer), ("classifier", logreg)])
clf_pipeline.fit(X_train, y_train)

pred = ['ham' if x==0 else 'spam' for x in clf_pipeline.predict([test])]

print(*pred)

Ваше тестовое сообщение: Have you visited the last lecture on physics?
ham
